# チョコボールのエンゼルの出現確率を推定する

In [ ]:
# ライブラリのインポート
import sys, os
import numpy as np
import pandas as pd
import scipy.stats as stats
import pymc3 as pm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# プロットする図を綺麗にする
sns.set()

## データを確認する
エンゼルの有無は`angel`列に記録されており、数値との関係は以下の通り
- 0 : エンゼル無し
- 1 : 銀のエンゼル
- 2 : 金のエンゼル

`campaign`列に開催中のキャンペーンIDが記録されている。
キャンペーンID=1の商品は、金のエンゼルの出現確率は2倍だが、銀のエンゼルの出現確率は0%である。

In [ ]:
# データの読み込み
data_raw = pd.read_csv('../data/chocoball_raw.csv')
tastes = pd.read_csv('../data/choco_tastes.csv')

print('data_raw.shape:', data_raw.shape)
print('tastes.shape:', tastes.shape)

In [ ]:
data_raw.head()

In [ ]:
tastes

エンゼルの出現数を確認する

In [ ]:
data_raw.groupby(['campaign', 'angel']).count()[['taste']]

銀のエンゼルの出現確率を予測する際に、`campaign=1`のデータを使わないようにする。

In [ ]:
df_data = data_raw[data_raw['campaign']!=1]
print('df_data.shape:', df_data.shape)
df_data.groupby(['campaign', 'angel']).count()[['taste']]

In [ ]:
df_data.groupby(['angel']).count()[['taste']]

## 仮定
- エンゼルはランダムに入っている
- 時期やフレーバーに依って確率は変化しない


## 最尤推定でパラメータを推定する

### モデル設定
- (スライド参照)
- エンゼルの出現を二項分布でモデル化する
  - エンゼルの出現確率を$\theta$とする
  - チョコボールの購入数（試行数）を$n$とする
$$
f(k|\theta) = \binom{n}{k}\theta^{k}(1-\theta)^{n-k}
$$

### 最尤推定量の計算
- (スライド参照)
- 求めたいパラメータはエンゼルの出現確率である$\theta$
- 最尤推定では、対数尤度$\log{L(\theta|X)}$をパラメータ$\theta$で微分して0となる値を推定値とする

$$
L(\theta|X)=\prod^{N}_{i=1}{f(k|\theta)} = \binom{n}{k}\theta^{k}(1-\theta)^{n-k} \\
\log{L(\theta|X)} = \log{\binom{n}{k}\theta^{k}(1-\theta)^{n-k}} \\
\qquad\qquad\qquad\qquad\qquad = \log{\binom{n}{k}} + \theta\log{{k}} + (1-\theta)\log{{n-k}} \\
$$

微分して0となるパラメータ

$$
\frac{d\log{L(\theta|X)}}{d\theta} = 0 \\
\hat{\theta} = \frac{k}{n}
$$

ということで、長々と数式を展開してきたが、結局は標本平均となる。

In [ ]:
theta_l = np.sum(df_data['angel']) / np.size(df_data['angel'].values)
print('estimated value(MLP):', theta_l)

### 推定値の活用
統計モデリングができたら、その結果を活用して様々なことができる。

#### 推定値をパラメータとした二項分布を確認
二項分布は試行数nを所与として、何回あたりを引くかの分布。

In [ ]:
ks = np.arange(0, 20, 1)
n = 100

# 確率質量関数を計算
pmf = stats.binom.pmf(ks, n, p=theta_l)

# Plot
plt.scatter(ks, pmf, label='n_sample={}'.format(n))
plt.legend()
plt.xlabel('k (number of angel)')
plt.ylabel('probability')
plt.savefig('binom_mle.png')

#### 何個買えばエンゼルが5個当たるのか？
- 二項分布を使って、当たりの数を所与として、尤度を算出することは可能
- 上記の推定を確率分布で考える場合、負の二項分布を利用する（ベルヌーイ試行）
  - $k$ : 成功数（エンゼルの出現数）
  - $x$ : k回成功するまでの失敗回数
  - $\theta$ : 1回の成功確率（エンゼルの出現確率）
$$
f(x|\theta) = \binom{k+x-1}{x}\theta^{k}(1-\theta)^{x}
$$

In [ ]:
k = 5
xs = np.arange(k+0, k+300, 1)

# 確率分布の計算
pmf_nb = stats.nbinom.pmf(xs, k, theta_l)
cdf_nb = pmf_nb.cumsum()

# 累積確率が50%を超える位置を算出
first_over_50 = list(cdf_nb).index(cdf_nb[cdf_nb>0.5].min())

# plot
fig = plt.figure(figsize=(13, 4))
ax = fig.subplots(1, 2)
ax[0].plot(xs, pmf_nb)
ax[0].set_title('Probability Mass Function')
ax[0].set_xlabel('False Count')
ax[0].set_ylabel('Probability Mass')
ax[1].plot(xs, cdf_nb)
ax[1].set_title('Cumulative Probability Mass Function')
ax[1].set_xlabel('False Count')
ax[1].set_ylabel('Cum. Probability')
ax[1].set_ylim([0.0, 1.1])
ax[1].vlines(x=first_over_50, ymin=0, ymax=1.0, color="red", label="50% Over")
print('50% Over point:{}, ({}+{})'.format(first_over_50 + k, first_over_50, k))
plt.savefig('purchase_number_mle.png')

#### エンゼルの出現をシミュレーション
n_trial回チョコボールを開封した時に、エンゼルが幾つ出るかのシミュレーション。  
意外と少ない場合(2個とか3個)も発生し得る。

In [ ]:
n_sample = 1000
n_trial = 100  # チョコボールを開ける個数

# シミュレーションの実行
with pm.Model() as model_angel:
    x = pm.Binomial('x', n=n_trial, p=theta_l)
    trace = pm.sample(n_sample, chains=1)

# plot
b = np.arange(0, max(trace['x'])+1, 1)
plt.hist(trace['x'], bins=b)
plt.xlabel('n_angel')
plt.savefig('simulation_mle.png')

## ベイズ推定でパラメータを推定する

### モデル設定
- (スライド資料参照)
- ベイズの式を思い出す
    - $p(\theta | x) \propto p(x | \theta)p(\theta)$
    - x : n個のチョコボールを開封して出たエンゼルの数
    - $\theta$ : 確率分布のパラメータ（エンゼルの含有率）
    - 尤度$p(x | \theta)$と事前分布$p(\theta)$を設定する必要がある
- 尤度関数
    - 最尤推定と同様に、二項分布を利用
    - $p(x | \theta) = \binom{n}{x}\theta^x(1-\theta)^{N-x}$
- 事前分布
    - ベータ分布
    - $p(\theta) = \frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha)\Gamma(\beta)}\theta^{\alpha-1}(1-\theta)^{\beta-1}$


#### ベータ分布の形状を見てみる

In [ ]:
params = [0.1, 1, 2, 10] # alpha, betaの例

x = np.linspace(0, 1, 100) # x軸の設定

fig = plt.figure(figsize=(13, 10))
ax = fig.subplots(len(params), len(params), sharex=True, sharey=True)
cnt=0
for i in range(len(params)):
    for j in range(len(params)):
        # パラメータalphaとbetaを設定
        a = params[i]
        b = params[j]
        # ベータ分布の確率密度を計算
        y = stats.beta(a, b).pdf(x)
        # plot
        ax[i, j].plot(x, y)
        ax[i, j].plot(0, 0, label="$\\alpha$ = {:3.2f}\n$\\beta$ = {:3.2f}".format(a, b), alpha=0)
        ax[i, j].legend()
        if i == (len(params)-1):
            ax[i,j].set_xlabel('$\\theta$')
        if j == 0:
            ax[i,j].set_ylabel('$p(\\theta)$')
plt.savefig('beta_dist_var.png')

### 解析的な計算方法(共役事前分布)

ベイズの定理を再度思い出す。
事後分布は尤度関数と事前分布の積に比例するという式である。
$$
p(\theta | y) \propto p(y | \theta)p(\theta)
$$
尤度関数には、二項分布で事前分布はベータ分布と定義したので、
ベイズの定理は以下のような式になる。
$$
p(\theta | y) \propto \frac{N!}{y!(N-y)!}\theta^y(1-\theta)^{N-y}\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha)\Gamma(\beta)}\theta^{\alpha-1}(1-\theta)^{\beta-1}
$$

$\theta$に関係しない部分は比例定数として押し込めてしまうことで、
以下の式が得られる。

$$
p(\theta | y)　\propto \theta^{\alpha-1+y}(1-\theta)^{\beta-1+N-y}
$$

この式はベータ分布に一致する。
$$
p(\theta | y)　= Beta(\alpha_{prior}+y, \beta_{prior}+N-y)
$$

つまり、今回のモデル定義においては、解析的に事後分布を導くことができた。  
このように尤度関数との積が同じ関数になる事前分布を「共役事前分布」と呼ぶ。
共役事前分布をモデルに利用すれば解析的に解を求めることができるが、
もっと複雑なモデルを使う場合には、一般的に解析解が得られない。

### 数値的な計算方法(MCMC)

#### 計算の実行
複雑なモデルや共役でない事前分布を使う場合、計算が困難か解析的には計算が不可能な場合がある。  
このような場合にも事後分布を計算するアルゴリズムとして、
マルコフチェーンモンテカルロ（MCMC）と呼ばれるアルゴリズムがある。  
詳細は省略するが、ざっくりとしたイメージでは、形状がわからない確率分布（事後分布）の大きさに比例してデータをサンプルするアルゴリズムである。

In [ ]:
d_angel = df_data['angel'].values
n_sample = 10000

with pm.Model() as model:
    # 事前分布
    #theta = pm.Beta('theta', alpha=1, beta=1)
    theta = pm.Uniform('theta', lower=0, upper=1)
    # 尤度
    y = pm.Binomial('y', n=len(d_angel), p=theta, observed=sum(d_angel))
    # sample
    trace = pm.sample(n_sample, chains=4)

pm.traceplot(trace)


In [ ]:
pm.plot_posterior(trace)

In [ ]:
d_angel = df_data['angel'].values
n_sample = 10000

with pm.Model() as model_angel:
    # 事前分布の設定
    #theta = pm.Uniform('theta', lower=0, upper=1) # 一様分布
    theta = pm.Beta('theta', alpha=1.0, beta=1.0) # 一様分布
    #theta = pm.Beta('theta', alpha=1, beta=2) # 上記ベータ分布の例参照
    
    # 尤度関数の設定
    obs = pm.Binomial('obs', n=len(d_angel), p=theta, observed=sum(d_angel))
    #obs = pm.Bernoulli('obs', p=theta, observed=d_angel) # ベルヌーイ分布
    
    # MCMCサンプルを得る
    trace = pm.sample(n_sample, chains=4)

#### 結果の解釈

推定対象である、二項分布のパラメータ$\theta$(エンゼルの含有率)の事後分布を確認する。
なお、以下の図はchain数（MCMCサンプル系列の数）分の結果が同時に表示されている。
- 左図:$\theta$の事後分布
- 右図:$\theta$のサンプル系列。ランダムにサンプルされていることが望ましい。

In [ ]:
fig = plt.figure(figsize=(8, 3))
ax = fig.subplots(1,2)
ax = ax[np.newaxis, :]

pm.traceplot(trace, ax=ax)
ax[0,0].vlines(x=theta_l, ymin=0, ymax=10, color="red", label="MLE")
ax[0,1].hlines(y=theta_l, xmin=0, xmax=n_sample, color="red", label="MLE")

plt.savefig('trace_plot_angel_rate.png')

次に、統計量を確認する。

In [ ]:
pm.summary(trace)

- mean:事後分布の期待値
- sd:サンプルの標本標準偏差
- mc_error:サンプリングに起因する誤差の推定値
- hpd_2.5:95%信用区間の下限
- hpd_97.5:95%信用区間の上限
- n_eff:サンプルサイズの効果量
- Rhat:chain間の分散とchain内の分散の比のようなもの。1に近いほど良い。大きい場合は、収束していないchainがある。

chainを全て統合して、事後分布を推定。
- 信用区間(HPD)をalpha_levelで指定

In [ ]:
pm.plot_posterior(trace, kde_plot=True, alpha_level=0.05)
plt.savefig('posterior_angel_rate.png')

#### 何個買えばエンゼルが5個当たるのか？
- 最尤推定の場合と同様に負の二項分布を利用して推定する

In [ ]:
theta_tr = trace['theta']
alpha_level = 0.05
k = 5
xs = np.arange(k+0, k+300, 1)
pmf_nb_ex = stats.nbinom.pmf(xs, k, theta_tr.mean())
pmf_nb_lb = stats.nbinom.pmf(xs, k, np.percentile(theta_tr, (alpha_level*50.0)))
pmf_nb_ub = stats.nbinom.pmf(xs, k, np.percentile(theta_tr, (100.0-alpha_level*50.0)))
cdf_nb_ex = pmf_nb_ex.cumsum()
cdf_nb_lb = pmf_nb_lb.cumsum()
cdf_nb_ub = pmf_nb_ub.cumsum()

ex_p = 0.5
first_over_ex = list(cdf_nb_ex).index(cdf_nb_ex[cdf_nb_ex>=ex_p].min())
first_over_lb = list(cdf_nb_lb).index(cdf_nb_lb[cdf_nb_lb>=ex_p].min())
first_over_ub = list(cdf_nb_ub).index(cdf_nb_ub[cdf_nb_ub>=ex_p].min())

fig = plt.figure(figsize=(13, 4))
ax = fig.subplots(1, 2)

ax[0].plot(xs, pmf_nb_ex)
ax[0].set_title('Probability Mass Function')
ax[0].set_xlabel('False Count')
ax[0].set_ylabel('Probability Mass')
ax[1].plot(xs, cdf_nb_ex)
ax[1].fill_between(xs, cdf_nb_lb, cdf_nb_ub, facecolor='y',alpha=0.5)
ax[1].set_title('Cumulative Probability Mass Function')
ax[1].set_xlabel('False Count')
ax[1].set_ylabel('Cum. Probability')
ax[1].set_ylim([0.0, 1.1])
ax[1].vlines(x=first_over_ex, ymin=0, ymax=1.0, color="red", label="{}% Over (ex)".format(ex_p))
ax[1].vlines(x=first_over_lb, ymin=0, ymax=1.0, color="green", label="{}% Over (lb)".format(ex_p))
ax[1].vlines(x=first_over_ub, ymin=0, ymax=1.0, color="blue", label="{}% Over (ub)".format(ex_p))
print('{}% Over point:{} ~ {} ~ {} (alpha_level={})'.format(ex_p*100, first_over_ub, first_over_ex, first_over_lb, alpha_level))
plt.savefig('purchase_number_bayes.png')